# Views and Copies

In our previous reading, we talked about how we could not just look at subsets of vectors, but also store those subsets in a new variable. For example, we could pull the middle entry of one vector and assign it to a new vector:

In [1]:
import numpy as np
a = np.array([42, 47, -1])
a

array([42, 47, -1])

In [2]:
new = a[1]
new

47

At the time, we illustrated what was happening with this set of pictures:

![vector_subsetting1](../week_2/img/C2W2-30_subsetting_part1.png)
![vector_subsetting2](../week_2/img/C2W2-30_subsetting_part2.png)
![vector_subsetting3](../week_2/img/C2W2-30_subsetting_part3.png)

And that was *close* to the truth about what was going on, but it wasn't quite the *full* truth. 

The reality is that when we create a subset in numpy and assign it to a new variable, what is *actually* happening is not that the variable is being assigned a *copy* of the values being the subset, but rather the variable is being assigned a *reference* to the subset, something that looks more like this:

[new image with referring arrow]

When numpy creates a reference to a subset of an existing array, that reference is called a *view*, because it's not a copy of the data in the original array, but an easy way to referring back to the original array -- it provides a *view* onto a subset of the original array. 

Why is this distinction important? It's important because it means that both variables -- `a` and `new` are actually both referencing the same data, and so changes make through one variable will propagate to the other. 

To illustrate in more detail, let's create two new vectors: `my_vector` and `my_subset`, where `my_subset` (as the name implies) is just a subset of `my_vector`:

In [3]:
my_vector = np.array([1, 2, 3, 4])
my_vector

array([1, 2, 3, 4])

In [4]:
my_subset = my_vector[1:3]
my_subset

array([2, 3])

[NEED ANOTHER PIC HERE?]

Now suppose we change the first entry of `my_subset` to be `-99`:

In [5]:
my_subset[0] = -99

Since the first entry in `my_subset` is just a reference to the second entry in `my_vector`, the change I made to `my_subset` will also propagate to `my_vector`:

In [6]:
my_vector

array([  1, -99,   3,   4])

And just as edits to `my_subset` will propagate to `my_vector`, so too will edits to `my_vector` propagate forward to `my_subset`:

In [7]:
my_vector[2] = 42
my_subset

array([-99,  42])

## A Symmetric Relationship

An important thing to emphasize that last point: when a view is created, the changes to *either* array may impact the other!

While we can say that numpy is "returning a view" when we subset an array, and we draw a picture which suggests that there is an "original array" and a "view", the fact that numpy creates views actually creates a *symmetric dependency* between the arrays involved. Yes, one may be "the original", but once a view has been created, changes to *either* array have the potential to propagate to the other: changes to the subset may impact the original, and changes to the original can impact the subset (if they impact the portion of the array referenced by the subset). 

## Why? Why Would Numpy Do This?

It is not uncommon, when they are first introduced to this behavior, for students to feel a little betrayed by numpy. "*Why*," they ask, "why would numpy do something that makes it so much harder to keep track of the consequences of changes I make to my data?"

The short answer, as with most things in numpy, is that it's all about speed. Creating a new copy of the data contained in the subset of a vector takes time, and so creating *views* instead of copies makes numpy faster.

## When do I get a view, and when do I get a copy?

Because numpy will *usually* create views when you subset a vector, and changes to views will propagate to the vectors associated with other variables, it's really important to keep track of when the object you're working with is a copy. 

Which brings us to the next slightly frustrating thing about numpy: the way that you ask for a subset will determine whether you get a view or a copy.


### Views and Copies from Subsetting

Generally speaking, **numpy will give you a *view* if you use simple indexing to get a subset of an array,** but it will provide a *copy* if you use any other methods. Recall that simple indexing is when you pass a single index, or a range of indices separated by a `:`. So `my_vector[2]` is simple indexing, and so is `my_vector[2:4]`.

So, for example, this simple indexing returns a view:

In [8]:
my_array = np.array([1, 2, 3])
my_subset = my_array[1:3]
my_subset

array([2, 3])

In [9]:
my_subset[0] = -1
my_array

array([ 1, -1,  3])

But if you ask for a subset any other way—such as with "fancy indexing" (where you pass a list when making your slice) or Boolean subsetting—you will NOT get a view, you will get a copy. As a result, changes made to your subset will not propagate back to `my_array`:

In [10]:
my_array = np.array([1, 2, 3])
my_subset = my_array[[1,2]]
my_subset[0] = -1
my_array

array([1, 2, 3])

In [11]:
my_array = np.array([1, 2, 3])
my_slice = my_array[my_array >= 2]
my_slice[0] = -1
my_array

array([1, 2, 3])

### Views and Copies When Editing

We established above that numpy will only return a view when you subset with simple indexing, but not when you use fancy indexing or Boolean subsetting. 

But it's also important to understand what types of modifications of a view will result in changes that propagate back to the original array. 

But if you modify a view with a simple indexing on the left-hand side of the assignment indicator (e.g., `my_subset[0] = ...` or `my_subset[0:2] = ...`), that change will propagate back to the original array (`my_array`).

But if we modify our vector and assign it to `my_subset` *without* that simple indexing on the left-hand side of the assignment operator, numpy will actually just create a new vector and assign it to the variable, not modify entries in our current vector. So in the following example, when numpy sees `my_subset * 2` it just creates a new vector with values equal to double the values in `my_subset`, then assigns that vector to the variable `my_subset`—it doesn't modify the data originally associated with `my_subset` (which is the same data underlying `my_array`):

In [12]:
my_array = np.array([1, 2, 3])
my_subset = my_array[1:3]
my_subset = my_subset * 2
my_subset

array([4, 6])

In [13]:
my_array

array([1, 2, 3])

If you want ever do want to do a full-array manipulation and preserve your view, you can just use square brackets on the left side of the assignment operator with just `:`:

In [14]:
my_array = np.array([1, 2, 3])
my_subset = my_array[1:3]
my_subset[:] = my_subset * 2
my_subset

array([4, 6])

In [15]:
my_array

array([1, 4, 6])

## Making a Copy

Of course, this type of propagating behavior is not always desirable, and so if one wishes to pull a subset of a vector (or array) that is a full copy and not a view, one can just use the `.copy()` method:

In [16]:
my_vector = np.array([1, 2, 3, 4])
my_subset = my_vector[1:3].copy()
my_subset

array([2, 3])

In [17]:
my_subset[0] = -99
my_subset

array([-99,   3])

In [18]:
my_vector

array([1, 2, 3, 4])

## Checking If An Array is a View or Copy

In all the examples above, I showed whether we were working with a view or a copy by making an edit to the array in question and then looking at whether the original data had changed. But surely there's a more direct way to check if an array is a view or a copy, right?

Well... kind of. There is a way to identify when an array references data that was not created at the moment that array was created. Thus, in a sense, you can identify views. And we detail how that works below. **But:** crucially, this method is only able to identify arrays that reference other data, **not** whether the array is **sharing data with another array.** In other words, while we know (from our discussion above) that the creation of a view is best thought of as creating a mutual relationship between two arrays, the tool numpy provides to interrogate an array's view-copy status does not respect the mutuality of this relationship. 

### .base

All numpy arrays have a property called `.base` which points back to the original data referenced by the array if the array is a view (it is referencing data that wasn't created when it was created). If an array is not a view of some other array, then `.base` returns `None`.

In [23]:
my_vector = np.array([1, 2, 3, 4])
my_vector.base is None

True

Given that, we can define a simple function that tells us if an array is a view or not:

In [25]:
def array_is_a_view(x):
    if x.base is None:
        print("Not a view!")
    else:
        print("This is a view.")

array_is_a_view(my_vector)

Not a view!


Now we can use this function to confirm that simple indexing gets views:

In [27]:
array_is_a_view(my_vector[1:2])

This is a view.


And fancy indexing does not:

In [28]:
array_is_a_view(my_vector[[2, 1]])

Not a view!


### Views and Copies in Practice

But while this trick is useful for confirming your logic of what does and does not return a view, its usefulness in practice is a little limited.

That's because the reason we care about views and copies is because **we are interested in knowing whether the changes that we make to an array may propagate to other arrays.** And that's not something this function can really tell us. 

The reason is twofold. First, when a view is created, from numpy's perspective the newly created array is a view on the old array, while the original array is *not* a view:

In [31]:
original_vector = np.array([1, 2, 3])
new_vector = original_vector[0:2]

array_is_a_view(original_vector)

Not a view!


In [32]:
array_is_a_view(new_vector)

This is a view.


But as we've discussed repeatedly, that doesn't mean that changes to `original_vector` won't impact other arrays, and that changes to other arrays (e.g., `new_vector`) might not impact `original_vector`. In other words, this trick tells us something about how numpy has implemented the view, but not about *the existence of relationships between arrays*.

The second reason is that you can have things that, from an implementation perspective, are views, but where there is *no* danger of changes to the array impacting anything else. This actually happens a lot when you re-assign a subset to the original variable:

In [35]:
my_vector = np.array([1, 2, 3, 4])
array_is_a_view(my_vector)

Not a view!


In [36]:
my_vector = my_vector[2:4]
array_is_a_view(my_vector)

This is a view.


Here we created an array with values 1, 2, 3, and 4, then subset the third and fourth entries. This subset is a view of the original data, and so `.base` is not `None`. But because we assigned the subset back to the vector `my_vector`, while that original array of 1, 2, 3, and 4 still exists in memory, nothing can access it except through the variable `my_vector`, and so you don't have to worry about any changes elsewhere in your code propagating to `my_vector` even if it technically is a view.

So in conclusion, yes, numpy has some tools to tell you about what is a view and what's a copy *from an implementation perspective*. But given that what matters from the perspective of a data scientist is whether changes to an array may propagate to other arrays, or changes to other arrays may propagate back, that tooling isn't particularly helpful.